In [39]:
import fitz
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import sys
import pandas as pd

In [40]:
# !pip install PyMuPDF

In [41]:
pdf_path = "data\\Datos_abiertos_historicos_etv_2022.pdf"

In [42]:
def extract_links(pdf_path):
    links = []
    doc = fitz.open(pdf_path)    
    page = doc[0]
    for link in page.get_links():
        links.append(
            link['uri']
        )
    doc.close()
    return links

In [43]:
def descompresor(links):
    carpeta = 'data\csv-files'
    success = 0
    fail_download = 0
    fail_process = 0
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    ruta_bitacora = 'data\\bitacora.txt'
    with open(ruta_bitacora, 'w') as log_file:

        #Redirigir la salida estándar al archivo
        sys.stdout = log_file
        for link in links:
            try:
                respuesta = requests.get(link)
                if respuesta.status_code == 200:
                    with ZipFile(BytesIO(respuesta.content)) as archivo_zip:
                        archivo_zip.extractall(carpeta)

                        # Cambiar el nombre del archivo
                        nombre_archivo = link[-32:-4]+'.csv'

                        for archivo in archivo_zip.namelist():
                            nuevo_nombre = os.path.join(carpeta, nombre_archivo)
                            os.rename(os.path.join(carpeta, archivo), nuevo_nombre)

                        print(f'Archivo descomprimido desde {link} en {carpeta}')
                        success+=1
                else:
                    print(f'Error al descargar {link}. Código de estado: {respuesta.status_code}')
                    fail_download+=1

            except Exception as e:
                print(f'Error al procesar {link}: {str(e)}')
                fail_process+=1
        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")
        
        #Restaurar la salida estándar al archivo
        sys.stdout = sys.__stdout__

        print(f"Hubo {success} exitos, {fail_download} fallas al descargar, {fail_process} fallas al descomprimir")

    print(f'Los resultados se han guardado en {ruta_bitacora}')

In [44]:
enlaces = extract_links(pdf_path)

descompresor(enlaces)

In [45]:
cvs_files_path = 'data\csv-files'

In [46]:
def get_dataframe(carpeta):
    dataframe = pd.DataFrame()
    # Recorrer todos los archivos en la carpeta
    for archivo_csv in os.listdir(carpeta):
        if archivo_csv.endswith('.csv'):
            ruta_completa = os.path.join(carpeta, archivo_csv)
            df = pd.read_csv(ruta_completa)
            dataframe = pd.concat([dataframe, df])
    return dataframe

In [47]:
dengue_2022 = get_dataframe(cvs_files_path)
dengue_2022

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2022-08-29,949565,1,26,28,21,2,2,28,21,...,2,2,2,2,5.0,2,5,1,28,21
1,2022-08-29,953022,2,37,28,21,2,2,28,21,...,2,2,2,2,5.0,2,5,1,28,21
2,2022-08-29,952991,1,40,14,38,2,2,14,6,...,2,2,2,2,5.0,1,5,3,14,38
3,2022-08-29,952244,1,24,14,6,2,2,14,6,...,2,2,2,2,5.0,1,5,3,14,6
4,2022-08-29,945141,2,25,14,6,2,2,14,6,...,2,2,2,2,5.0,1,5,3,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,2022-03-28,933199,1,23,27,4,2,2,27,4,...,2,2,2,2,5.0,1,5,3,27,4
3116,2022-03-28,933202,1,13,7,28,2,2,7,28,...,2,2,2,2,5.0,2,5,1,7,28
3117,2022-03-28,933212,1,18,14,21,2,2,14,21,...,2,2,2,2,5.0,1,5,3,14,21
3118,2022-03-28,933218,1,32,14,21,2,2,14,21,...,2,2,2,2,5.0,1,5,3,14,21


In [48]:
dengue_2022['FECHA_ACTUALIZACION'].unique() #Se indentifican formatos

array(['2022-08-29', '2022-11-28', '2022-05-30', '2022-01-31',
       '2022-02-28', '2022-10-31', '2022-08-01', '2022-05-02',
       '2022-01-06', '2022-10-10', '2022-04-04', '2022-07-04',
       '2022-09-05', '2022-12-05', '2022-06-06', '2022-02-08',
       '2022-03-07', '2022-11-07', '2022-08-09', '2022-05-09',
       '2022-01-10', '2022-04-11', '2022-07-11', '2022-09-13',
       '2022-12-12', '2022-06-13', '2022-02-14', '2022-03-14',
       '2022-11-14', '2022-08-15', '2022-05-16', '2022-01-17',
       '2022-10-17', '2022-04-18', '2022-07-18', '2022-09-21',
       '2022-06-20', '2022-03-22', '2022-11-22', '2022-08-22',
       '2022-05-23', '2022-01-24', '2022-10-24', '25/04/2022',
       '25/07/2022', '2022-09-26', '2022-06-27', '2022-03-28'],
      dtype=object)

In [49]:
#Se cambian a objetos datetime los formatos mixtos
date1 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%Y-%m-%d')
date2 = pd.to_datetime(dengue_2022['FECHA_ACTUALIZACION'], errors='coerce', format='%d/%m/%Y')
dengue_2022['FECHA_ACTUALIZACION'] = date1.fillna(date2)
#Se organizan las fechas
dengue_2022 = dengue_2022.sort_values(by='FECHA_ACTUALIZACION')

In [50]:
dengue_2022

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
25930,2022-01-06,910390,2,36,25,17,2,2,25,1,...,2,2,2,2,5.0,1,5,1,25,17
22132,2022-01-06,897037,1,44,24,28,2,2,24,28,...,2,2,2,2,5.0,1,5,3,24,28
22133,2022-01-06,897063,1,0,14,22,2,2,14,22,...,2,2,2,2,5.0,2,5,1,14,22
22134,2022-01-06,897876,1,37,20,67,2,2,20,67,...,2,2,2,2,5.0,2,5,1,20,67
22135,2022-01-06,897246,1,9,17,17,2,2,17,17,...,2,2,2,2,5.0,1,1,2,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14303,2022-12-12,948486,1,44,16,102,2,2,16,102,...,2,2,2,2,5.0,1,5,3,16,102
14304,2022-12-12,948487,2,30,34,334,2,2,7,101,...,2,2,2,2,5.0,1,5,1,99,999
14305,2022-12-12,948489,2,5,15,82,2,2,15,82,...,2,2,2,2,5.0,1,5,3,15,82
14292,2022-12-12,956756,2,53,15,82,2,2,15,82,...,2,2,2,2,5.0,2,5,1,15,82
